In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# 코드에서 nms제외해야함

In [2]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



fatal: detected dubious ownership in repository at '/mnt/nas2/users/eslim/workspace/OpenPCDet'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/nas2/users/eslim/workspace/OpenPCDet


In [3]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_head_rev')==False:
    os.mkdir('./deploy_pillar_sfaw_head_rev')
log_file = './deploy_pillar_sfaw_head/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-04-16 11:28:17,342   INFO  Loading Custom dataset.
2024-04-16 11:28:17,343   INFO  Total samples for CUSTOM dataset: 0


In [4]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-04-16 11:28:19,827   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-16 11:28:20,800   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-16 11:28:20,827   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [5]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
inputs = batch_dict


In [6]:
model

CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [7]:
model.dense_head

CenterHead(
  (shared_conv): Sequential(
    (0): Conv2d(384, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (heads_list): ModuleList(
    (0): SeparateHead(
      (center): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (center_z): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [8]:
inputs_vfe = model.vfe(inputs)
inputs_3d = model.backbone_3d(inputs_vfe)
inputs_bev = model.map_to_bev_module(inputs_3d)
inputs_2d = model.backbone_2d(inputs_bev)
inputs_head = model.dense_head(inputs_2d)

# inputs_bev
inputs_head

{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1,
 'pillar_features': tensor([[-1.0969e+00, -8.8788e-02,  1.5693e-01,  ..., -1.0477e+00,
           3.8000e-02, -1.9860e-01],
         [-1.0964e+00, -7.0831e-02,  1.6052e-01,  ..., -1.0824e+00,
           4.2364e-02, -1.9235e-01],
         [-1.0862e+00, -1.0094e-01,  1.7857e-01,  ..., -1.0550e+00,
           2.6313e-02, -1.5948e-01],
         .

In [10]:
# data_dict = {}
input_dict = {}
input_dict['data_dict']={}
# input_dict['data_dict']['voxel_features'] = inputs_2d['voxel_features']
# input_dict['data_dict']['use_lead_xyz'] = inputs_2d['use_lead_xyz']
# input_dict['data_dict']['batch_size'] = torch.tensor(inputs_2d['batch_size'])
# input_dict['data_dict']['pillar_features'] = inputs_2d['pillar_features']
# input_dict['data_dict']['voxel_coords'] = inputs_2d['voxel_coords']
# input_dict['data_dict']['spatial_features'] = inputs_2d['spatial_features']
input_dict['data_dict']['spatial_features_2d'] = inputs_2d['spatial_features_2d']
# input_dict['data_dict']['final_box_dicts'] = inputs_2d['final_box_dicts']

# data_dict.update(inputs_bev)
# data_dict

In [11]:
# DSVT_2d_Backbone = model.backbone_2d
# DSVT_2d_Backbone

In [12]:
with torch.no_grad():
    DSVT_2d_Backbone = model.backbone_2d
    DSVT_head = model.dense_head
    # dsvtblocks_list = DSVT_Backbone.stage_0
    # layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    # inputs = model.vfe(inputs)
    # voxel_info = DSVT_Backbone.input_layer(inputs)
    # set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]

# model.dense_head

In [14]:
input_names = [
    # 'voxel_features',
    # 'use_lead_xyz',
    # 'batch_size',
    # 'pillar_features',
    # 'voxel_coords',
    # 'spatial_features',
    'spatial_features_2d'
]
output_names = ["output",]

In [15]:
base_name = "./deploy_pillar_sfaw_head_rev/dsvt_head"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

# allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()

torch.onnx.export(
    DSVT_head,
    # alldsvtblockstrt_inputs,
    input_dict,
    onnx_path, 
    input_names=input_names,
    output_names=output_names, 
    # dynamic_axes=dynamic_axes,
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



KeyError: 'batch_size'

In [ ]:
# # test onnx
# ort_session = ort.InferenceSession(onnx_path)
# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# # compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
#               ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
#               ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
#               ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
#               ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
#               ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
# ort_outs = ort_session.run(None, ort_inputs)
# ort_outs[0].shape
